In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import random

In [2]:
USER_AGENT_LIST = ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36']
DRIVER_FILE_PATH = "/Users/qunishdash/.wdm/drivers/chromedriver/mac64/116.0.5845.96/chromedriver-mac-x64/chromedriver"

In [3]:
def get_chrome_driver(headless_flag):
    # Set up the Selenium webdriver
    service = Service(executable_path=DRIVER_FILE_PATH)
    chrome_options = Options()

    if headless_flag:
        # in case you want headless browser
        chrome_options.add_argument("--headless")
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-gpu")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("user-agent={}".format(random.choice(USER_AGENT_LIST)))

    else:
        # in case  you want to open browser
        chrome_options.add_argument("--start-maximized")
        chrome_options.add_argument("user-agent={}".format(random.choice(USER_AGENT_LIST)))
        chrome_options.headless = False

    driver = webdriver.Chrome(service=service, options=chrome_options) 

    return driver

In [4]:
from pymongo import MongoClient

client = MongoClient()
database_name = 'bigbasket_data_db'
collection_name = 'gourmet-world-food-9-lv'
db = client[database_name]
collection = db[collection_name]

driver = get_chrome_driver(False)

cursor = collection.find({}, {'product_url': 1})
product_urls = [doc['product_url'] for doc in cursor]

/var/folders/fh/yk5wd4g14xd7qzb16xnjp2ww0000gn/T/ipykernel_4040/97347481.py:19: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = False


In [5]:
# collection_names = [
#     "fruits-vegetables-lv",                   #done
#     "foodgrains-oil-masala-lv",                   #done
#     "bakery-cakes-dairy-lv",                   #done
#     "beverages-lv",                   #done
#     "snacks-branded-foods-1lv",                   #done
#     "snacks-branded-foods-2lv",                   #done
#     "snacks-branded-foods-3lv",                   #done
#     "snacks-branded-foods-4lv",                   #done
#     "snacks-branded-foods-5lv",                   #done
#     "snacks-branded-foods-6lv",                   #done
#     "snacks-branded-foods-7lv",                   #done
#     "snacks-branded-foods-8lv",                   #done
#     "snacks-branded-foods-9lv",                   #done
#     "snacks-branded-foods-10lv",                   #done
#     "beauty-hygiene-1lv",                   #done
#     "beauty-hygiene-2lv",                   #done
#     "beauty-hygiene-3lv",                   #done
#     "beauty-hygiene-4lv",                   #done
#     "beauty-hygiene-5lv",                   #done
#     "beauty-hygiene-6lv",                   #done
#     "beauty-hygiene-7lv",                   #done
#     "beauty-hygiene-8lv",                   #done
#     "beauty-hygiene-9lv",                   #done
#     "cleaning-household-lv",                   #done
#     "kitchen-garden-pets-lv",                   #done
#     "kitchen-garden-pets-1lv",                   #done
#     "kitchen-garden-pets-2lv",                   #done
#     "kitchen-garden-pets-3lv",                   #done
#     "kitchen-garden-pets-4lv",                   #done
#     "kitchen-garden-pets-5lv",                   #done
#     "kitchen-garden-pets-6lv",                   #done
#     "kitchen-garden-pets-7lv",                   #done
#     "kitchen-garden-pets-8lv",                   #done
#     "kitchen-garden-pets-9lv",                   #done
#     "eggs-meat-fish-lv",                   #done
#     "gourmet-world-food-1-lv",                   #done
#     "gourmet-world-food-2-lv",                   #done
#     "gourmet-world-food-3-lv",                   #done
#     "gourmet-world-food-4-lv",                   #done
#     "gourmet-world-food-5-lv",                   #done
#     "gourmet-world-food-6-lv",                   #done
#     "gourmet-world-food-7-lv",                   #done
#     "gourmet-world-food-8-lv",                   #...
#     "gourmet-world-food-9-lv",                   #
#     "gourmet-world-food-10-lv",                   #done
#     "baby-care-lv"                   #done
# ]

In [6]:
def get_product_data(driver, collection_url):
    product_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/h1"))
    )
    product_id = collection_url.split("/")[4]
    try:
        product_name = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/section[1]/h2").text
    except Exception as e:
        product_name = ''
    try:
        product_rating = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[3]/div/div/div[1]/div[2]/div/div/div[1]/div[1]").text
    except Exception as e:
        product_rating = ''
    try:
        product_path = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[1]/div[1]').text
    except Exception as e:
        product_path = ''
    try:
        product_brand = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/a').text
    except Exception as e:
        product_brand = ''
    
    additional_data = {}
    additional_data_xpath_template = "/html/body/div[1]/div/div[2]/section[1]/div/div[{}]"
    n = 1

    while True:
        xpath = additional_data_xpath_template.format(n)

        try:
            section_element = driver.find_element(By.XPATH, xpath)
            key_value_parts = section_element.text.split('\n', 1)
            
            if len(key_value_parts) >= 2:
                key = key_value_parts[0]
                value = key_value_parts[1]
            else:
                key = "Additional Data"  # Set a default label for the key
                value = key_value_parts[0] if len(key_value_parts) == 1 else None

            if key != "More About The Product From Brand" and key != "From the Brand":
                additional_data[key] = value
            n += 1  # Move to the next element
        except NoSuchElementException:
            break 
                
            additional_data[key] = value
            n += 1  # Move to the next element
        except NoSuchElementException:
            continue
            
    new_dict = {}
    if "Other Product Info" in additional_data:
        other_product_info_text = additional_data["Other Product Info"]
        lines = other_product_info_text.split('\n')
        other_product_info = {}
        for line in lines:
            if ':' in line:
                key, value = line.split(':', 1)
                other_product_info[key.strip()] = value.strip()
        
        new_dict = {
            key: value for key, value in other_product_info.items()
        }

    result = {
        "product_url": collection_url,
        "product_id": product_id,
        "product_name": product_name,
        "product_rating": product_rating,
        "product_path": product_path,
        "product_brand": product_brand,
        **{k: v for k, v in additional_data.items() if k != "Other Product Info"},
        **new_dict
    }

    return result

In [7]:
def get_variant_data(driver):
    try:
        try:
            product_name = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/section[1]/h2").text
        except Exception as e:
            product_name = ''
        variant_name = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/h1').text.replace(product_name + ", ", "")
    except Exception as e:
        variant_name = ''
    try:
        variant_discount_price = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/div/div/table/tbody/tr[2]/td[2]").text
    except Exception as e:
        variant_discount_price = ''
    try:
        variant_original_price = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/div/div/table/tbody/tr[1]/td[2]").text
    except Exception as e:
        variant_original_price = ''
    try:
        variant_discount_percentage = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/div[1]/div/table/tbody/tr[3]/td[2]').text.replace("(Inclusive of all taxes)", "")
    except Exception as e:
        variant_discount_percentage = ''
    try:
        variant_cost_per_unit = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/div[1]/div/div/table/tbody/tr[2]/td[4]").text
    except Exception as e:
        variant_cost_per_unit = ''
    try:
        variant_shade = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[3]/section[2]/div[2]/span").text
    except Exception as e:
        variant_shade = ''

    variant_images_xpath_template = "/html/body/div[1]/div/div[2]/div[2]/div[2]/div[2]/div/div[{}]/img"
    max_variants = 10
    
    variant_images_list = []
    
    for n in range(1, max_variants + 1):
        variant_images_xpath = variant_images_xpath_template.format(n)
        
        try:
            variant_images_element = driver.find_element(By.XPATH, variant_images_xpath)
            variant_images_src = variant_images_element.get_attribute("src")
            variant_images_list.append(variant_images_src)
            
        except NoSuchElementException:
            break
        except Exception as e:
            print(f"An error occurred for Variant {n}:", e)

    result = {
        "variant_name": variant_name,
        "discount_price": variant_discount_price,
        "original_price": variant_original_price,
        "discount_percentage": variant_discount_percentage,
        "cost_per_unit": variant_cost_per_unit,
        "variant_images_list": variant_images_list
    }
    return result

In [8]:
max_variants = 15
error_urls = []

mongo_host = 'localhost'
mongo_port = 27017
pv_database_name = 'bigbasket_all_data_db'
pv_collection_name = 'gourmet-world-food-pv'

driver = webdriver.Chrome()

client = MongoClient(mongo_host, mongo_port)
db_pv = client[pv_database_name]
collection_pv = db_pv[pv_collection_name]

scraped_count = 0
total_urls = len(product_urls)

for collection_url in product_urls:
    existing_data = collection_pv.find_one({"product_url": collection_url})
    if existing_data:
        # print(f"Data already exists for {collection_url}. Skipping scraping and insertion.")
        scraped_count += 1
        continue

    driver.get(collection_url)
    
    results = []
    variant_found = False
    
    for n in range(1, max_variants + 1):
        variant_xpath = f'/html/body/div[1]/div/div[2]/div[2]/div[3]/section[2]/div[2]/div[{n}]'

        try:
            variant_button = driver.find_element(By.XPATH, variant_xpath)
            variant_button.click()
            
            product_data = get_product_data(driver, collection_url)
            variant_data = get_variant_data(driver)
            
            combined_data = {**product_data, **variant_data}
            results.append(combined_data)
            
        except NoSuchElementException:
            # Variant not found, exit the loop or handle as needed
            product_data = get_product_data(driver, collection_url)
            variant_data = get_variant_data(driver)
            combined_data = {**product_data, **variant_data}
            results.append(combined_data)
            break
            
        except Exception as e:
            print(f"An error occurred for Variant {n}", collection_url)
            error_urls.append(collection_url) 
            
            product_data = get_product_data(driver, collection_url)
            variant_data = get_variant_data(driver)
            combined_data = {**product_data, **variant_data}
            results.append(combined_data)
            break
            
    try:
        collection_pv.insert_many(results)
        # print(f"Data inserted successfully into MongoDB for {collection_url}.")
    except Exception as e:
        print(f"Error inserting data for {collection_url}: {e}")
        
    print(f"Scraped URLs: {scraped_count}/{total_urls}")
    scraped_count += 1

Scraped URLs: 1907/2155
Scraped URLs: 1944/2155
Scraped URLs: 1945/2155
Scraped URLs: 1946/2155
Scraped URLs: 1947/2155
Scraped URLs: 1948/2155
Scraped URLs: 1949/2155
Scraped URLs: 1950/2155
Scraped URLs: 1951/2155
Scraped URLs: 1952/2155
Scraped URLs: 1953/2155
Scraped URLs: 1954/2155
Scraped URLs: 1955/2155
Scraped URLs: 1956/2155
Scraped URLs: 1957/2155
Scraped URLs: 1958/2155
Scraped URLs: 1959/2155
Scraped URLs: 1960/2155
Scraped URLs: 1961/2155
Scraped URLs: 1962/2155
Scraped URLs: 1963/2155
Scraped URLs: 1964/2155
Scraped URLs: 1965/2155
Scraped URLs: 1966/2155
Scraped URLs: 1967/2155
Scraped URLs: 1968/2155
Scraped URLs: 1969/2155
Scraped URLs: 1970/2155
Scraped URLs: 1971/2155
Scraped URLs: 1972/2155
Scraped URLs: 1973/2155
Scraped URLs: 1974/2155
Scraped URLs: 1975/2155
Scraped URLs: 1976/2155
Scraped URLs: 1977/2155
Scraped URLs: 1978/2155
Scraped URLs: 1979/2155
Scraped URLs: 1980/2155
Scraped URLs: 1981/2155
Scraped URLs: 1982/2155
Scraped URLs: 1983/2155
Scraped URLs: 19

In [9]:
if scraped_count == total_urls:
    print("FINISHED!!!!", f"{scraped_count}/{total_urls}")

FINISHED!!!! 2155/2155


In [10]:
print("URLs with errors:", error_urls)

URLs with errors: []


In [11]:
driver.quit()
client.close()